# タスク 1a: テキスト生成の実行

このノートブックでは、サポートエンジニアから受けたカスタマーサービスの品質について否定的なフィードバックを提供した顧客に対して、大規模言語モデル (LLM) を使用してメール返信を生成する方法を学習します。このノートブックでは、顧客の以前のメールに基づいて、お礼のメモ付きのメールを生成します。Amazon Nova Lite モデルは、Boto3 クライアントで Amazon Bedrock API で使用します。

このタスクで使用されるプロンプトは、ゼロショットプロンプトと呼ばれます。ゼロショットプロンプトでは、タスクまたは目的の出力を平易な言葉で言語モデルに説明します。その後、モデルは事前にトレーニングされた知識と機能を使用して、提供されたプロンプトのみに基づいて応答を生成したり、タスクを完了したりします。

#### シナリオ
あなたはAnyCompanyのカスタマーサービスマネージャーの Bob です。お客様の中には、カスタマーサポートエンジニアによるサービスに満足していない方がいらっしゃり、サービスについて否定的なフィードバックをされています。今、あなたはそうしたお客様に対し、謙虚にサービスのご不満をお詫びし、信頼回復したいと考えています。LLMを活用することで、お客様との過去のメールのやり取りからお客様の感情を汲み取った上で、人間らしくパーソナライズされたメールを一括で生成する必要があります。

## タスク 1a.1: 環境のセットアップ

このタスクでは、環境をセットアップします。

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## タスク 1a.2: テキストの生成

このタスクでは、Amazon Bedrock サービスがメールを生成するための入力を準備します。

In [ ]:
# create the prompt
prompt_data = """
指示: AnyCompany のカスタマーサービスマネージャーである Bob から、当社のカスタマーサポートエンジニアが提供したサービスについて否定的なフィードバックを提供した顧客「John Doe」にメールを書いてください。"""

In [ ]:
body = json.dumps({
    "messages": [
        {
            "role": "user",
            "content": [{"text": prompt_data}]
        }
    ],
    "inferenceConfig": {
        "maxTokens": 8192,
        "stopSequences": [],
        "temperature": 0,
        "topP": 0.9
    }
}) 

次に、Amazon Nova Lite を呼び出します。

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **注:** Amazon Nova Lite は、約 300kトークンのコンテキストウィンドウをサポートし、次のパラメータを受け入れます。:
- `messages`: ロールとコンテンツを持つメッセージオブジェクトの配列
- `inferenceConfig`: これらは、モデルが出力を生成する際に考慮するパラメータです。

Amazon Bedrock API は、次のものを受け入れる API `invoke_model` を提供します:
- `modelId`: これは、Amazon Bedrock で使用できるさまざまな基盤モデルのモデル ARN です
- `accept`: 入力リクエストのタイプ
- `contentType`: 出力のコンテンツ タイプ
- `body`: プロンプトと構成で構成される JSON 文字列

使用可能なテキスト生成モデル ID については、[ドキュメント](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) を参照してください。

## タスク 1a.3: Amazon Nova Lite 言語モデルを呼び出す

このタスクでは、以前に作成したプロンプトに基づいてモデルが出力を生成する方法を調べます。

### 完全な出力生成

このメールは、入力リクエストを理解し、さまざまなモダリティに対する固有の理解を活用して、Amazon Nova Lite モデルを使用して生成されます。この　API　は同期リクエストのため、モデルによって出力全体が生成されるまで待機します。

In [ ]:
#invoke model
modelId = 'amazon.nova-lite-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"
try:

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body.get('output').get('message').get('content')[0].get('text')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error


In [ ]:
# Print the complete response from Nova Lite
print(outputText)


### ストリーミング出力生成

Bedrock では、モデルによってチャンクの形式で生成される出力をストリーミングすることもできます。このメールは、ストリーミング オプションを使用してモデルを呼び出すことによって生成されます。`invoke_model_with_response_stream` は、読み取り可能な `ResponseStream` を返します。

In [ ]:
# invoke model with response stream
output = []
try:
    
    response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
    stream = response.get('body')
    
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                if 'contentBlockDelta' in chunk_obj:
                    text = chunk_obj['contentBlockDelta']['delta']['text']
                    output.append(text)
                    print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')
                    i+=1
            
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

レスポンス付きストリーム アプローチは、モデルの出力をすばやく取得するのに役立ち、ユーザーが読みながらサービスが出力を完了できるようにします。これは、モデルに長いテキストを生成するように要求するユースケースに役立ちます。後で、生成されたすべてのチャンクを組み合わせて完全な出力を形成し、ユースケースに使用できます。

In [ ]:
#combine output chunks
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
complete_output = ''.join(output)
print(complete_output)


これで、Amazon Bedrock API を使用するためのの基本的な boto3 SDK を試しました。この API を使用して、顧客の否定的なフィードバックに応答するメールを生成するユースケースを確認しました。

### 試してみましょう
- 特定のユースケースに合わせてプロンプトを変更し、さまざまなモデルの出力を評価します。
- トークンの長さを変えることで、サービスのレイテンシと応答性がどのように変化するかを理解します。
- さまざまなプロンプトエンジニアリングの原則を適用して、より良い出力を取得します。

### クリーンアップ

あなたはこのノートブックを完了しました。ラボの次のパートに移るには、下記を実行してください。:

- このノートブックファイルを閉じ、 **Task1b_ja_jp.ipynb** を開きます。